<a href="https://colab.research.google.com/github/yoojinleee/CarGame/blob/main/CarGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pygame
import random
import sys
import time

# Initializing the game
pygame.init()

# Making the settings for the screen display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Car Racing Game")

# Frame rate for moving car
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 48)

#Car
car_width, car_height = 50, 100
car = pygame.Rect(WIDTH//2, HEIGHT - 120, car_width, car_height)

# Making the car move
car_speed = 5

#Obstacle
obstacle_width, obstacle_height = 50, 100
obstacles = []
obstacle_speed = 5

#Game Activity
running = True
game_active = True
last_math_time = time.time ()
math_interval = 30

#Colors
WHITE = (255, 255, 255)
Black = (0,0,0)
Red = (255, 0, 0)
Blue = (0, 0, 255)

def draw_car():
  pygame.draw.rect(screen, Blue, car)

def draw_obstacles():
  for obs in obstacles:
    pygame.draw.rect(screen, Red, obs)

def spawn_obstacle():
  x = random.randint(0, WIDTH - obstacle_width)
  new_obstacle = pygame.Rect(x, -obstacle_height, obstacle_width, obstacle_height)
  obstacles.append(new_obstacle)

def move_obstacles():
  for obs in obstacles:
    obs.y += obstacle_speed
  #Deleting obstacles off the screen
  obstacles[:] = [obs for obs in obstacles if obs.y < HEIGHT]

def check_collision():
  for obs in obstacles:
    if car.colliderect(obs):
      return True
  return False

def show_message(message):
  text = font.render(message, True, Black)
  rect = text.get_rect(center=(WIDTH//2, HEIGHT//2))
  screen.blit(text, rect)
  pygame.display.flip()
  pygame.time.delay(2000)

def start_math_quiz():
  global num1, num2, answer, user_input, game_state
  num1 = random.randint(1,10)
  num2 = random.randint(1,10)
  answer = num1 * num2
  user_input = ""
  game_state = "math_quiz"

In [7]:
pip install pygame


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
print(pygame.font.get_fonts())

['arialunicode', 'applesdgothicneo', 'menlo', 'stheitimedium', 'symbol', 'geneva', 'hiraginosansgb', 'lastresort', 'notoserifmyanmar', 'helveticaneue', 'sfnsmono', 'ヒラキノ角コシックw8', 'pingfang', 'noteworthy', 'notosansoriya', 'ヒラキノ角コシックw9', 'thonburiui', 'sfgeorgian', 'sfhebrew', 'optima', 'zapfdingbats', 'sfarmenian', 'applebraille', 'avenirnextcondensed', 'lucidagrande', 'courier', 'sfcompact', 'applebrailleoutline6dot', 'kohinoortelugu', 'applebraillepinpoint6dot', 'muktamahee', 'newyork', 'sfns', 'sfcamera', 'avenir', 'kohinoor', 'notosansmyanmar', 'sfarmenianrounded', 'ヒラキノ明朝pron', 'avenirnext', 'kohinoorgujarati', 'aquakana', 'palatino', 'monaco', 'applesymbols', 'notonastaliq', 'stheitilight', 'geezapro', 'ヒラキノ角コシックw7', 'times', 'adtnumeric', 'ヒラキノ丸コpronw4', 'ヒラキノ角コシックw6', 'ヒラキノ角コシックw4', 'notosanskannada', 'sfarabic', 'sfnsrounded', 'ヒラキノ角コシックw5', 'markerfelt', 'applebraillepinpoint8dot', 'keyboard', 'ヒラキノ角コシックw1', 'sfgeorgianrounded', 'sfcompactrounded', 'arialhb', 'applebrailleout

In [9]:
# Pausing the game every 30 seconds to ask a math question
# Generate two random numbers between 1 and 10

def ask_math_question():
    num1 = random.randint(1, 10)
    num2 = random.randint(1, 10)
    answer = num1 * num2 # Calculate the correct answer
    user_input = ""
    asking = True


# Begin the math question loop
    while asking:
        screen.fill(WHITE) # Clearing the screen so we can ask the math question
        question = font.render(f"What is {num1} x {num2}?", True, Black)
        input_box = font.render(user_input, True, Red)
        screen.blit(question, (WIDTH // 2 - question.get_width() // 2, HEIGHT // 2 - 50))
        screen.blit(input_box, (WIDTH // 2 - input_box.get_width() // 2, HEIGHT // 2 + 10))
        pygame.display.flip()

# Handling different events that could potentially happen
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_BACKSPACE:
                    user_input = user_input[:-1]
                elif event.key == pygame.K_RETURN:
                    if user_input.isdigit():
                        if int(user_input) == answer:
                            asking = False  # Resume the game
                        else:
                            show_message("Wrong! You died.")
                            return False  # Game is over
                else:
                    user_input += event.unicode
    return True  # Continue the game

game_state = "playing"
user_input = ""
num1, num2 = 0,0
answer = 0

In [ ]:
while running:
  clock.tick(60)
  screen.fill(WHITE)

  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      running = False

    if game_state == "math_quiz" and event.type == pygame.KEYDOWN:
      if event.key == pygame.K_BACKSPACE:
        user_input = user_input[:-1]
      elif event.key == pygame.K_RETURN:
        if user_input.isdigit():
          if int(user_input) == answer:
            game_state = "playing"
            last_math_time = time.time()
          else:
            show_message("Wrong! You died.")
            game_active = False
            game_state = "game_over"
      else:
        user_input += event.unicode

    if game_state == "math_quiz":
      question = font.render(f"What is {num1} x {num2}?", True, Black)
      input_box = font.render(user_input, True, Red)
      screen.blit(question, (WIDTH//2 - question.get_width()//2, HEIGHT//2-50))
      screen.blit(input_box, (WIDTH//2 - input_box.get_width()//2, HEIGHT//2+10))

    elif game_state == "playing" and game_active:
      keys = pygame.key.get_pressed()
      if keys[pygame.K_LEFT] and car.left>0:
        car.x -= car_speed
      if keys[pygame.K_RIGHT] and car.right<WIDTH:
        car.x += car_speed

      if random.randint(1,20)==1:
        spawn_obstacle()

      move_obstacles()
      draw_car()
      draw_obstacles()

      if check_collision():
        show_message("You crashed!")
        game_active = False
        game_state = "game_over"

    #Time-based math challenge
      if time.time() - last_math_time > math_interval:
        start_math_quiz()

    elif game_state == "game_over":
      show_message("Press R to restart")
      keys = pygame.key.get_pressed()
      if keys[pygame.K_r]:
          car.x = WIDTH//2
          car.y = HEIGHT - 120
          obstacles.clear()
          last_math_time = time.time()
          game_active = True
          game_state = "playing"

  pygame.display.flip() % DisplayHandle

pygame.quit() % DisplayHandle
sys.exit()

SystemExit: 

/Users/aliciakim/Library/Python/3.13/lib/python/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import pygame
import random
import sys
import time

# Initializing the game
pygame.init()

# Making the settings for the screen display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Car Racing Game")

# Frame rate for moving car
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 48)

#Colors
WHITE = (255, 255, 255)
Black = (0,0,0)
Red = (255, 0, 0)
Blue = (0, 0, 255)
Yellow = (255, 255, 0)

#Road Background
road_line_offset = 0 
line_height = 40 
line_spacing = 60 

#Car
car_width, car_height = 50, 100
car = pygame.Rect(WIDTH//2, HEIGHT - 120, car_width, car_height)

# Making the car move
car_speed = 5

#Obstacle
obstacle_width, obstacle_height = 40, 45
obstacles = []
obstacle_speed = 5

#Game Activity
running = True
game_active = True
game_state = "playing"
last_math_time = time.time ()
math_interval = 30

score = 0 

num1, num2 = 0,0
answer = 0
user_input = ""

def draw_background():
  global road_line_offset
#setting the colors 
  road_color = (50, 50, 50)
  grass_color = (34, 139, 34)
  line_color = Yellow
#creating the road boundaries
  road_width = 400 
  road_left = (WIDTH - road_width) // 2
  road_right = road_left + road_width 
#drawing the grass as the borders 
  pygame.draw.rect(screen, grass_color, (0, 0, road_left, HEIGHT))
  pygame.draw.rect(screen, grass_color, (road_right, 0, WIDTH - road_right, HEIGHT))

#drawing the road
  pygame.draw.rect(screen, road_color, (road_left, 0, road_width, HEIGHT))

#detailing the road with yellow lines running down the middle 
  num_lines = HEIGHT // (line_height + line_spacing) + 2
  for i in range(num_lines): 
    y = (i * (line_height + line_spacing)) + road_line_offset
    pygame.draw.rect(screen, line_color, (WIDTH//2 - 5, y % HEIGHT, 10, line_height))

   #scrolling effect for the road 
  road_line_offset += obstacle_speed
  road_line_offset %= (line_height + line_spacing) 

def draw_car():
  #car body 
  body_color = (0, 102, 204) 
  cabin_color = (180, 230 ,255)
  light_color = (255, 255, 204) 
  wheel_rim_color = (105, 105, 105) 
# main body of the car 
  pygame.draw.rect(screen, body_color, (car.x, car.y + 10, car.width, car.height - 10), border_radius=6)

  #windshield of the car 
  pygame.draw.rect(screen, cabin_color, (car.x + 10, car.y + 15, car.width - 20, 20), border_radius=4)
  #drawing the front  headlights 
  pygame.draw.rect(screen, light_color, (car.x + 5, car.y + car.height - 8, 10, 4))
  pygame.draw.rect(screen, light_color, (car.x + car.width - 15,  car.y + car.height - 8, 10, 4))

  #tail lights of the car 
  pygame.draw.rect(screen, (255, 0, 0), (car.x + 5, car.y + 12, 5 ,5))
  pygame.draw.rect(screen, (255, 0, 0), (car.x + car.width - 10, car.y + 12, 5 ,5))

  #wheels 
  wheel_radius = 6
  wheel_offset_x = 8
  front_wheel_y = car.y + car.height - 18
  back_wheel_y = car.y + 25

  pygame.draw.circle(screen, wheel_rim_color, (car.x + wheel_offset_x, front_wheel_y), wheel_radius + 2)
  pygame.draw.circle(screen, Black, (car.x + wheel_offset_x, front_wheel_y), wheel_radius) 
  
  pygame.draw.circle(screen, wheel_rim_color, (car.x + wheel_offset_x, back_wheel_y), wheel_radius + 2) 
  pygame.draw.circle(screen, Black, (car.x + wheel_offset_x, back_wheel_y), wheel_radius) 

  pygame.draw.circle(screen, wheel_rim_color, (car.x + car.width - wheel_offset_x, front_wheel_y), wheel_radius + 2) 
  pygame.draw.circle(screen, Black, (car.x + car.width - wheel_offset_x, front_wheel_y), wheel_radius)

  pygame.draw.circle(screen, wheel_rim_color, (car.x + car.width - wheel_offset_x, back_wheel_y), wheel_radius + 2) 
  pygame.draw.circle(screen, Black, (car.x + car.width - wheel_offset_x, back_wheel_y), wheel_radius)
 


def draw_obstacles():
  for obs in obstacles: #Loop through the obstacle list
    pygame.draw.rect(screen, Red, obs) #Draw a red obstacle on screen

def spawn_obstacle():
  road_left = (WIDTH - 400) // 2 #Calculate the left boundaries
  road_right = (WIDTH + 400) // 2 - obstacle_width #Calculate the right boundaries
  x = random.randint(road_left, road_right) #Randomly generate an X position in between the two boundaries
  new_obstacle = pygame.Rect(x, -obstacle_height, obstacle_width, obstacle_height) #Create a new obstacle/rectangle
  obstacles.append(new_obstacle) #Add the new obstacle to the obstacles list

def move_obstacles():
 global score
 for obs in obstacles: 
   obs.y += obstacle_speed #Move all obstacles down by increasing their y coordinate using obstacle_speed
   if obs.y >= HEIGHT: #If the obstcale moves off screen
    score += 1 #Increase the score by one
    obstacles.remove(obs) #Remove obstacle

def check_collision():
  for obs in obstacles:
    if car.colliderect(obs): #Check if the car collides with the obstacle
      return True #Return true if there is a collision
  return False #Return false if there is no collision

def show_message(message, delay=2000): 
  screen.fill(WHITE) #Clear the screen 
  text = font.render(message, True, Black)
  rect = text.get_rect(center=(WIDTH//2, HEIGHT//2))
  screen.blit(text, rect) #Display a message in the center 
  pygame.display.flip()
  pygame.time.delay(delay) #Wait for a specific time before resuming 

def start_math_quiz(): 
  global num1, num2, answer, user_input, game_state
  num1 = random.randint(1,10) #Random integer between 1-10
  num2 = random.randint(1,10) #Second random integer between 1-10
  answer = num1 * num2 #Set correct answer
  user_input = "" #Resent user input
  game_state = "math_quiz" #Switch game state to math_quiz

while running:
  clock.tick(30) #Limit loop to 30FPS 
  draw_background() 

  for event in pygame.event.get(): 
    if event.type == pygame.QUIT: #If the user presses quit, the code stops running
      running = False

    if game_state == "math_quiz":
      if event.type == pygame.KEYDOWN: 
        if event.key == pygame.K_BACKSPACE: #Deleting the input
          user_input = user_input[:-1]
        elif event.key == pygame.K_RETURN: #When the user presses enter
          if user_input.isdigit(): #When the user input is a digit
            if int(user_input) == answer: #and when the user input is the answer
              score += 5 #add to score
              game_state = "playing" #resume the game 
              last_math_time = time.time()
            else:
              show_message("Wrong! You died.") #When the user input is wrong show a message
              game_active = False 
              game_state = "game_over" #Switch game state to game over
        else:
          user_input += event.unicode

    elif game_state == "game_over": #When the game is over
      if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_r: #When the user presses R, restart the game
          car.x = WIDTH//2
          car.y = HEIGHT - 120
          obstacles.clear()
          last_math_time = time.time()
          game_active = True
          game_state = "playing"
          score = 0
        elif event.key == pygame.K_q: #When the user presses Q, quit the game
          pygame.quit()
          sys.exit()


  if game_state == "math_quiz":
#dimensions of the math quiz text box
    box_width = 400 
    box_height = 150
    box_x = WIDTH // 2 - box_width // 2
    box_y = HEIGHT // 2 - box_height // 2 

#creating a semi-transparent surface for the text box to clearly show the question
    quiz_box = pygame.Surface((box_width, box_height))
    quiz_box.set_alpha(220)
    quiz_box.fill(WHITE)
    screen.blit(quiz_box, (box_x, box_y))

    question = font.render(f"What is {num1} x {num2}?", True, Black) #Display the question with two randomly generated integers
    input_box = font.render(user_input, True, Red) #Set user input as red
    screen.blit(question, (WIDTH//2 - question.get_width()//2, HEIGHT//2-30)) #Position the texts
    screen.blit(input_box, (WIDTH//2 - input_box.get_width()//2, HEIGHT//2+20))

  elif game_state == "playing" and game_active:
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] and car.left > (WIDTH - 400) // 2: #Moves the car left within road boundaries
      car.x -= car_speed
    if keys[pygame.K_RIGHT] and car.right < (WIDTH + 400) // 2: #Moves the car right within road boundaries
      car.x += car_speed

    if random.randint(1,30)==1: #Randomly spawn obstacles (1 in 30 chance per frame)
      spawn_obstacle()

    move_obstacles() 
    draw_car()
    draw_obstacles()

    score_box_width = 180 
    score_box_height = 60
    score_box_x = 10 
    score_box_y = 10

    score_surface = pygame.Surface((score_box_width, score_box_height))
    score_surface.set_alpha(180)
    score_surface.fill((30, 30, 30))
    screen.blit(score_surface, (score_box_x, score_box_y))

    pygame.draw.rect(screen, Yellow, (score_box_x, score_box_y, score_box_width, score_box_height), 3)
    score_text = font.render(f"Score: {score}", True, WHITE) #Display the score
    screen.blit(score_text, (score_box_x + 15, score_box_y + 15))

    if check_collision(): #Check for collisions
      show_message("You crashed!") #If collisions happen, show the message
      game_active = False
      game_state = "game_over"

    #Time-based math challenge
    if time.time() - last_math_time > math_interval: #Trigger a math quiz after a certain interval from the last quiz
      start_math_quiz()
    
    pygame.display.flip()
    

  elif game_state == "game_over":
    screen.fill(WHITE) #Clear the screen
    msg = font.render("Game Over! Press R to restart or Q to quit", True, Black) #Display the game over message
    screen.blit(msg, (WIDTH//2 - msg.get_width()//2, HEIGHT//2))
    
  pygame.display.flip()

pygame.quit() #Exit the code
sys.exit()